In [1]:
!pip uninstall -y pandas -q
!pip install -q pandas==1.5.0
import pandas as pd

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
d

In [2]:
!pip install -q datasets==2.16.0
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
dask-cuda 23.8.0 requires 

In [3]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token ('your-hf-token')"

In [4]:
train = pd.read_csv("/kaggle/input/malawi-data/Train.csv")
train['Paragraph(s) Number'] = train['Paragraph(s) Number'].fillna(" ")

In [5]:
import torch
import transformers
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
from datasets import load_dataset
from peft import prepare_model_for_kbit_training,LoraConfig,PeftModel,get_peft_model

from trl import SFTTrainer

from transformers import set_seed

set_seed(42)

2024-03-03 11:28:41.137152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 11:28:41.137286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 11:28:41.269906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
device = "cuda" # the device to load the model onto
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
# Quantization configuration

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Loading the model and tokenizer

model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=600,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
def pre_process(row):
    ques = row['Question Text']
    ans = row['Question Answer']
    keywords = row['Keywords']
    res = f"Answer: {ans}\n\nKeywords: {keywords}"

    text = f"""[INST] Retrieve and present comprehensive and informative answers to the question using various reasoning strategies and evidence sources.
    Output should include :
    "Answer": The concise response to the question.
    "Keywords": A brief, keyword-rich text including most important keywords.

    This whole is a single question: {ques}
    {res}
    [/INST]
    """
    row['text'] = text
    return row

In [8]:
train = train.apply(pre_process, axis=1)

In [9]:
train.to_csv("train.csv", index=False)

In [10]:
finetuning_dataset_loaded = load_dataset("csv", data_files='./train.csv', split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

In [12]:
project = "finetuned-med-fp16"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

training_arguments=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        num_train_epochs=10,
        per_device_train_batch_size=1,
        learning_rate=2.5e-4, 
        logging_steps=700,

        optim="paged_adamw_32bit",
        lr_scheduler_type="cosine",
        logging_dir="./logs",        
        save_strategy="steps",     
        save_steps=700,                
        report_to='none',          
    )

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=finetuning_dataset_loaded,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 600
  warnings.warn(


Map:   0%|          | 0/748 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
700,0.714100
1400,0.418600
2100,0.250100
2800,0.168500
3500,0.119200
4200,0.088700
4900,0.064700
5600,0.052700
6300,0.044600
7000,0.041600


TrainOutput(global_step=7480, training_loss=0.18623028821486204, metrics={'train_runtime': 12761.41, 'train_samples_per_second': 0.586, 'train_steps_per_second': 0.586, 'total_flos': 5.883083420049408e+16, 'train_loss': 0.18623028821486204, 'epoch': 10.0})